In [1]:
import sqlite3
import pandas as pd
from tqdm import tqdm

In [2]:
all = pd.read_csv('../static/data/covid/전국 모든정보.csv',sep=',',encoding='utf8')
del all['Unnamed: 0']
all.head(3)

,일자,지역,사망자,확진자,전일대비,격리해제,격리중,지역발생,해외유입,10만명당
0,2020-03-04,제주,0,3,0,0,0,0,0,0.4
1,2020-03-04,경남,0,65,1,0,0,0,0,1.9
2,2020-03-04,경북,6,774,89,0,0,0,0,29.1


In [3]:
all.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5709 entries, 0 to 5708
Data columns (total 10 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   일자      5709 non-null   object 
 1   지역      5709 non-null   object 
 2   사망자     5709 non-null   int64  
 3   확진자     5709 non-null   int64  
 4   전일대비    5709 non-null   int64  
 5   격리해제    5709 non-null   int64  
 6   격리중     5709 non-null   int64  
 7   지역발생    5709 non-null   int64  
 8   해외유입    5709 non-null   int64  
 9   10만명당   5709 non-null   float64
dtypes: float64(1), int64(7), object(2)
memory usage: 446.1+ KB


In [4]:
conn = sqlite3.connect('covid.db')
cur = conn.cursor()

# 모든 정보
- 테이블 생성

In [5]:
sql = '''create table sido (
    date text not null,
    city text not null,
    death int default 0,
    increase int default 0,
    casenum int default 0,
    local int default 0,
    inflow int default 0,
    isolation int default 0, 
    release int default 0,
    per100k real default 0.00
);'''

cur.execute(sql)
conn.commit()

- 값 정제 <br>
int64는 int32로, float64는 float32로

In [6]:
text_col = list(all.iloc[0,:2])
text_col

['2020-03-04', '제주']

In [7]:
int_col = list(all.iloc[0,2:-1])
int_col

[0, 3, 0, 0, 0, 0, 0]

In [8]:
float_col = list(all.iloc[0,-1:])
float_col

[0.4]

In [9]:
text_col = list(all.iloc[0,:2])
for i in range(2,9):
    text_col.append(int(all.iloc[0,i]))
text_col.append(float(all.iloc[0,-1]))
text_col  

['2020-03-04', '제주', 0, 3, 0, 0, 0, 0, 0, 0.4]

- 값 추가

In [10]:
len(all)

5709

In [11]:
sql = """ insert into sido values(?,?,?,?,?,?,?,?,?,?)
"""
for i in tqdm(range(len(all))):
    text_col = list(all.iloc[i,:2])
    for k in range(2,9):
        text_col.append(int(all.iloc[i,k]))
    text_col.append(float(all.iloc[i,-1]))
    params = text_col
    cur.execute(sql, params)
    conn.commit()

# 기존 일자별

In [13]:
daily = pd.read_csv('../static/data/covid/전국코로나일별.csv',sep=',',encoding='utf8')
daily.head(3)

,date,서울,부산,대구,인천,대전,광주,울산,세종,경기,강원,충북,충남,경북,경남,전북,전남,제주,합계
0,2020-03-01,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,2020-03-02,10,3,469,0,0,0,3,0,7,8,1,13,67,4,1,0,0,586
2,2020-03-03,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


- 테이블 만들기

In [14]:
sql_table = '''create table daily (
  ilja text not null primary key,
  seoul int default 0, busan int default 0, daegu int default 0,
  incheon int default 0, daejeon int default 0, gwangju int default 0,
  ulsan int default 0, sejong int default 0, gyeonggi int default 0,
  gangwon int default 0, chungbuk int default 0, chungnam int default 0,
  gyeongbuk int default 0, gyeongnam int default 0, jeonbuk int default 0,
  jeonnam int default 0, jeju int default 0, 
  gye int default 0);'''
cur.execute(sql_table)
conn.commit()

- 값 넣기

In [15]:
sql_insert = 'insert into daily values(?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?);'
for i in range(len(daily)):
    params = [daily.iloc[i,0]]
    for k in range(1,19):
        params.append(int(daily.iloc[i,k]))
    cur.execute(sql_insert, params)
    conn.commit()

In [16]:
cur.close()
conn.close()